# Taxi problem

Follow the tutorial: https://www.learndatasci.com/tutorials/reinforcement-q-learning-scratch-python-openai-gym/

In [10]:
import gym
from IPython.display import clear_output
from time import sleep
import time
import numpy as np
import random

In [2]:
env = gym.make("Taxi-v3").env
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



The filled square represents the taxi, which is yellow without a passenger and green with a passenger.

The pipe ("|") represents a wall which the taxi cannot cross.

R, G, Y, B are the possible pickup and destination locations. The blue letter represents the current passenger pick-up location, and the purple letter is the current destination.

In [4]:
env.reset() # reset environment to a new, random state
env.render()

print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

Action Space Discrete(6)
State Space Discrete(500)


In [5]:
state = env.encode(3, 1, 2, 0) # (taxi row, taxi column, passenger index, destination index)
print("State:", state)

env.s = state
env.render()

State: 328
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



# Random Policy

In [6]:
env.s = 328  # set environment to illustration's state

epochs = 0
penalties, reward = 0, 0

frames = [] # for animation

done = False

while not done:
    action = env.action_space.sample()
    state, reward, done, info = env.step(action)

    if reward == -10:
        penalties += 1
    
    # Put each rendered frame into dict for animation
    frames.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action': action,
        'reward': reward
        }
    )

    epochs += 1    

Timesteps taken: 507
Penalties incurred: 157


In [12]:
def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(.1)

#visialize 1 episode
print_frames(frames)   
print("Timesteps taken: {}".format(epochs))
print("Penalties incurred: {}".format(penalties))

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 507
State: 0
Action: 5
Reward: 20


In [8]:
"""Evaluate random policy"""
total_epochs = 0
total_penalties = 0
episodes = 100
episode_reward_list = []
episode_reward = 0

tic = time.perf_counter()
#loop through 100 episodes
for _ in range(episodes):
    #set current state to starting environment
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0
    
    #done = successfully dropped off passenge
    done = False

    #while not successfully drop off passenger
    while not done:
        #select a random action
        action = env.action_space.sample()
        state, reward, done, info = env.step(action)
        episode_reward += reward

        #set penalty based on reward value for action
        if reward == -10:
            penalties += 1
        #increment episode
        epochs += 1

    #after 1 episode, sum penalties and episodes
    total_penalties += penalties
    total_epochs += epochs
    episode_reward_list.append(episode_reward)
    episode_reward = 0

toc = time.perf_counter()
average_time = (toc - tic)/episodes
#print evaluation of agents performance
print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")
print(f"Average reward per timestep: {sum(episode_reward_list) / total_epochs}")
print(f"Average time per episode: {average_time:0.4f} seconds")

Results after 100 episodes:
Average timesteps per episode: 2449.9
Average penalties per episode: 796.72
Average reward per timestep: -3.918274215274093
Average time per episode: 0.0217 seconds


# Q-learning

In [11]:
q_table = np.zeros([env.observation_space.n, env.action_space.n])

In [12]:
%%time
"""Training the agent"""
# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1

# For plotting metrics
all_epochs = []
all_penalties = []

for i in range(1, 100001):
    state = env.reset()

    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(q_table[state]) # Exploit learned values

        next_state, reward, done, info = env.step(action) 
        
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value #update new q value

        if reward == -10:
            penalties += 1

        state = next_state #move to next state
        epochs += 1
        
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")

print("Training finished.\n")

Episode: 100000
Training finished.

Wall time: 30 s


In [13]:
q_table[328]

array([ -2.40490521,  -2.27325184,  -2.41021105,  -2.36095895,
       -10.08774199, -10.06496847])

In [15]:
"""Evaluate agent's performance after Q-learning"""

total_epochs, total_penalties = 0, 0
episodes = 100
episode_reward_list = []
episode_reward = 0
tic = time.perf_counter()
for _ in range(episodes):
    state = env.reset() #reset the environment
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action) #select an action
        episode_reward += reward

        if reward == -10:
            penalties += 1

        epochs += 1
    #after 1 episode, sum penalties and episodes
    total_penalties += penalties
    total_epochs += epochs
    episode_reward_list.append(episode_reward)
    episode_reward = 0

toc = time.perf_counter()
average_time = (toc - tic)/episodes
#print evaluation of agents performance
print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")
print(f"Average reward per timestep: {sum(episode_reward_list) / total_epochs}")
print(f"Average time per episode: {average_time:0.4f} seconds")

Results after 100 episodes:
Average timesteps per episode: 12.82
Average penalties per episode: 0.0
Average reward per timestep: 0.6380655226209049
Average time per episode: 0.0002 seconds
